# K - Nearest Neigbours

In [1]:
import random
import math
import numpy as np
import pandas as pd
from collections import Counter
from typing import List, Tuple, TypeVar

In [2]:
X = TypeVar("X") 
Y = TypeVar("Y")

In [3]:
def raw_majority_vote(labels:List[str]) -> str:
    votes = Counter(labels)
    print(votes)
    winner, _ = votes.most_common(1)[0]
    return winner

In [4]:
winner = raw_majority_vote(["a", "b", "a", "b", "c"])
print(f"Winner = {winner}")

Counter({'a': 2, 'b': 2, 'c': 1})
Winner = a


In case of **ties** in majority votes, we have 3 options:

1. Pick one winner at random
2. Weight the votes by distance and pick the weighted winner        (Outlier could be a possible near neighbour)
3. reduce the k until we find a winner

we wiil implement the third, considering the wights asigned in 2nd

In [5]:
def majority_vote(labels: List[str]) -> str:
    """assume that labels are ordered from nearest to furthest"""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winner = len([count for count in vote_counts.values() if count == winner_count])
    if num_winner == 1:
        return winner                        # Only one winner
    else:
        return majority_vote(labels[:-1])   # call again without the farthest

In [6]:
winner = majority_vote(["a", "b", "c", "a", "b"])
print(f"Winner = {winner}")

Winner = a


### Pre-reqs

In [7]:
Vector = List[float]

In [8]:
def subtract(v: Vector, w: Vector) -> Vector:
    assert len(v) == len(w)
    diff = [v_i - w_i for v_i, w_i in zip(v, w)]
    return diff

In [9]:
def dot_product(v: Vector, w: Vector) -> float:
    # asser lenth of V and W are equal
    assert len(v) == len(w), "length of vectors are not equal"
    # compute vector product
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

In [10]:
# squared sum of vectors
def sum_of_squares(v: Vector) -> float:
    # compute squared sum finding dot product of V aginst V itself
    return dot_product(v, v)

In [11]:
# compute magnitude of a vector
def magnitude(v: Vector) -> float:
    # compute magnitude which is the square root of sum of squares of elemnts in given vector
    return math.sqrt(sum_of_squares(v))

In [12]:
# distance between 2 vectors
def distance(v: Vector, w: Vector) -> float:
    # compute distance by using subtract vector function and magnitude function
    return magnitude(subtract(v, w))

### Function def

In [13]:
from typing import NamedTuple

In [14]:
class LabeledPoint(NamedTuple):
    point: Vector
    label: str


def knn_classify(k: int, labeled_points: List[LabeledPoint], new_point: Vector) -> str:
    """Classify the new_point based on k nearest neighbours of labeled_points"""

    # Order the labeled points from nearest to furthest
    by_distance = sorted(labeled_points, key=lambda lp: distance(lp.point, new_point))

    # Find the labels for the K closest
    k_nearest_labels = [lp.label for lp in by_distance[:k]]

    # Let them vote!
    return majority_vote(k_nearest_labels)

## Iris Classification Model using KNN with UCI dataset

Dataset: https://archive.ics.uci.edu/dataset/53/iris

sepal_length, sepal_width, petal_length, petal_width, class

Eg:- 5.1, 3.5, 1.4, 0.2, iris-setos

We're going to build a model to predict a flowers class based on its sepal and petal dimensions

In [15]:
from typing import Dict
# import csv
from collections import defaultdict

In [16]:
def parse_iris_row(row: list[str]) -> LabeledPoint:
    """sepal_length, sepal_width, petal_length, petal_width, class"""
    measurements = [float(value) for value in row[:-1]]
    label = row[-1]
    return LabeledPoint(measurements, label)

Testing with a small example

In [17]:
labeled_point = parse_iris_row(["5.1", "3.5", "1.4", "0.2", "iris-setos"])
print(f"out_put: {labeled_point}")

out_put: LabeledPoint(point=[5.1, 3.5, 1.4, 0.2], label='iris-setos')


Testing with original data using our KNN classifier

Load Data:

In [18]:
# Reading with pandas
import pandas as pd

In [19]:
df = pd.read_csv("data\iris.data", names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
print(df.shape)
df.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [21]:
df1 = df.astype('str')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sepal_length  150 non-null    object
 1   sepal_width   150 non-null    object
 2   petal_length  150 non-null    object
 3   petal_width   150 non-null    object
 4   class         150 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [22]:
iris_data = [parse_iris_row(row) for row in df1.values]
iris_data

[LabeledPoint(point=[5.1, 3.5, 1.4, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.9, 3.0, 1.4, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.7, 3.2, 1.3, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.6, 3.1, 1.5, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[5.0, 3.6, 1.4, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[5.4, 3.9, 1.7, 0.4], label='Iris-setosa'),
 LabeledPoint(point=[4.6, 3.4, 1.4, 0.3], label='Iris-setosa'),
 LabeledPoint(point=[5.0, 3.4, 1.5, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.4, 2.9, 1.4, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.9, 3.1, 1.5, 0.1], label='Iris-setosa'),
 LabeledPoint(point=[5.4, 3.7, 1.5, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.8, 3.4, 1.6, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[4.8, 3.0, 1.4, 0.1], label='Iris-setosa'),
 LabeledPoint(point=[4.3, 3.0, 1.1, 0.1], label='Iris-setosa'),
 LabeledPoint(point=[5.8, 4.0, 1.2, 0.2], label='Iris-setosa'),
 LabeledPoint(point=[5.7, 4.4, 1.5, 0.4]

spliting data

In [23]:
def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Split the given data into fractions of prob given [[prob, 1-prob]]"""  # prob is fractio of training data set
    data = data[:]                   # Making a copy
    random.shuffle(data)
    cut = int(len(data) * prob)
    return data[:cut], data[cut:]

In [24]:
data = [n for n in range(1000)]

# some time we will have paired input and output variables. so we need to make sure to put corresponding values together either training or testing dataset
X = TypeVar("X") 
Y = TypeVar("Y")


def train_test_split(
    xs: List[X], ys: List[Y], test_pct: float
) -> Tuple[List[X], List[X], List[Y], List[Y]]:
    """Split them by using indeices"""

    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct)

    return (
        [xs[i] for i in train_idxs],
        [xs[i] for i in test_idxs],
        [ys[i] for i in train_idxs],
        [ys[i] for i in test_idxs],
    )  # x train, x test, y train, y test

In [25]:
# iris_data = [parse_iris_row(["5.1", "3.5", "1.4", "0.2", "iris-setos"])]        #ignore if reading from above data set
random.seed(12)

iris_train, iris_test = split_data(iris_data, 0.70)

Testing the KNN model

In [26]:
# Track how many times our model did correct predictions
from typing import Tuple


confusion_matrix: Dict[Tuple[str, str], int] = defaultdict(int)
num_correct = 0

#using our model to predict over the processed training and testing data set

for iris in iris_data:
    # Choosing k = 5
    predicted = knn_classify(9, iris_train, iris.point)
    actual = iris.label

    if predicted == actual:
        num_correct += 1

    confusion_matrix[(predicted, actual)] += 1

pct_correct = ((num_correct/len(iris_data)) * 100)

print(f'Accuracy on testing data {pct_correct} %')


Accuracy on testing data 98.0 %
